## **GoogLeNet (InceptionV3) for BreaKHis Dataset - Breast Cancer Classification**
***
This code implements a **GoogLeNet-based deep learning model** (using `InceptionV3`) to classify histology images into **Benign** and **Malignant** categories from the **BreaKHis dataset**.

### **Key Steps:**
1. **Dataset Preparation**  
   - Loads image paths from the dataset directories (`benign/40X` and `malignant/40X`).
   - Assigns labels: **0 for Benign, 1 for Malignant**.
   - Splits the dataset into **60% training, 10% validation, and 30% testing**.
   - Converts images into a TensorFlow dataset with preprocessing:
     - Reads images as PNG.
     - Resizes to **224x224** (compatible with GoogLeNet).
     - Normalizes pixel values.

2. **Model Implementation (GoogLeNet - InceptionV3)**
   - Loads **InceptionV3** (pre-trained on ImageNet) **without the top layer**.
   - **Freezes pre-trained layers** to prevent overfitting.
   - Adds a custom classifier:
     - **GlobalAveragePooling2D** layer.
     - **Fully connected layer (128 neurons, ReLU activation).**
     - **Output layer (1 neuron, Sigmoid activation for binary classification).**
   - Compiles the model using **Adam optimizer** and **binary cross-entropy loss**.

3. **Training the Model**
   - Trains the model using the training dataset.
   - Validates on a separate validation set.

4. **Evaluating the Model**
   - Predicts on the test dataset.
   - Computes performance metrics:
     - **Accuracy**
     - **F1 Score**
     - **Geometric Mean (G-Mean)**
     - **Informedness (IBA)**

This model helps classify histology images into **benign or malignant tumors** using deep learning.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***
**Magnification Factor: 40X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import InceptionV3  # Using InceptionV3 (closest to GoogLeNet)
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/40X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/40X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total dataset
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

def prepare_dataset(images, labels, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(process_path)
    if shuffle:
        dataset = dataset.shuffle(1000)
    return dataset.batch(BATCH_SIZE)

train_dataset = prepare_dataset(train_images, train_labels, shuffle=True)
val_dataset = prepare_dataset(val_images, val_labels)
test_dataset = prepare_dataset(test_images, test_labels)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load GoogLeNet (InceptionV3) without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 626
Total Malignant Images: 1370
Training samples: 1197
Validation samples: 200
Testing samples: 599
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 229s 6s/step - accuracy: 0.5706 - loss: 0.7954 - val_accuracy: 0.7650 - val_loss: 0.4498
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 553ms/step - accuracy: 0.7658 - loss: 0.4956 - val_accuracy: 0.8250 - val_loss: 0.3661
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 621ms/step - accuracy: 0.8109 - loss: 0.4073 - val_accuracy: 0.9000 - val_loss: 0.3083
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 545ms/step - accuracy: 0.8547 - loss: 0.3315 - val_accuracy: 0.9000 - val_loss: 0.2813
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 559ms/step - accuracy: 0.8850 - loss: 0.2979 - val_accuracy: 0.8850 - val_loss: 0.2738
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 23s 839ms/step - accuracy: 0.8916 - loss: 0.2809 - val_accuracy: 0.9150 - val_loss: 0.2448
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 561ms/

***
**Magnification Factor: 100X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import InceptionV3  # Using InceptionV3 (closest to GoogLeNet)
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/100X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/100X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total dataset
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

def prepare_dataset(images, labels, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(process_path)
    if shuffle:
        dataset = dataset.shuffle(1000)
    return dataset.batch(BATCH_SIZE)

train_dataset = prepare_dataset(train_images, train_labels, shuffle=True)
val_dataset = prepare_dataset(val_images, val_labels)
test_dataset = prepare_dataset(test_images, test_labels)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load GoogLeNet (InceptionV3) without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 649
Total Malignant Images: 1437
Training samples: 1251
Validation samples: 209
Testing samples: 626
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 220s 7s/step - accuracy: 0.5785 - loss: 1.0862 - val_accuracy: 0.7608 - val_loss: 0.5527
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 573ms/step - accuracy: 0.7107 - loss: 0.5333 - val_accuracy: 0.7464 - val_loss: 0.5115
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 662ms/step - accuracy: 0.7868 - loss: 0.4701 - val_accuracy: 0.8086 - val_loss: 0.4271
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 556ms/step - accuracy: 0.8159 - loss: 0.4105 - val_accuracy: 0.8230 - val_loss: 0.3893
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 617ms/step - accuracy: 0.8190 - loss: 0.3936 - val_accuracy: 0.8325 - val_loss: 0.3745
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 575ms/step - accuracy: 0.8286 - loss: 0.3711 - val_accuracy: 0.8517 - val_loss: 0.3539
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 710ms/step - accuracy: 0.8407 - loss: 0.3456 - val_accura

***
**Magnification Factor: 200X**
***

In [3]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import InceptionV3  # Using InceptionV3 (closest to GoogLeNet)
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/200X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/200X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total dataset
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

def prepare_dataset(images, labels, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(process_path)
    if shuffle:
        dataset = dataset.shuffle(1000)
    return dataset.batch(BATCH_SIZE)

train_dataset = prepare_dataset(train_images, train_labels, shuffle=True)
val_dataset = prepare_dataset(val_images, val_labels)
test_dataset = prepare_dataset(test_images, test_labels)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load GoogLeNet (InceptionV3) without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 623
Total Malignant Images: 1390
Training samples: 1207
Validation samples: 202
Testing samples: 604
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 254s 6s/step - accuracy: 0.7219 - loss: 0.5838 - val_accuracy: 0.7822 - val_loss: 0.4239
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 541ms/step - accuracy: 0.8275 - loss: 0.4166 - val_accuracy: 0.8317 - val_loss: 0.3927
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 612ms/step - accuracy: 0.8450 - loss: 0.3718 - val_accuracy: 0.8366 - val_loss: 0.3959
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 491ms/step - accuracy: 0.8803 - loss: 0.3042 - val_accuracy: 0.8267 - val_loss: 0.3871
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 832ms/step - accuracy: 0.8768 - loss: 0.2929 - val_accuracy: 0.8267 - val_loss: 0.4059
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 604ms/step - accuracy: 0.9131 - loss: 0.2476 - val_accuracy: 0.8069 - val_loss: 0.3846
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 536ms/

***
**Magnification Factor: 400X**
***

In [4]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import InceptionV3  # Using InceptionV3 (closest to GoogLeNet)
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/400X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/400X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total dataset
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)  # Decode PNG images
    img = tf.image.resize(img, [224, 224])  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

def prepare_dataset(images, labels, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(process_path)
    if shuffle:
        dataset = dataset.shuffle(1000)
    return dataset.batch(BATCH_SIZE)

train_dataset = prepare_dataset(train_images, train_labels, shuffle=True)
val_dataset = prepare_dataset(val_images, val_labels)
test_dataset = prepare_dataset(test_images, test_labels)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load GoogLeNet (InceptionV3) without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier on top
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 588
Total Malignant Images: 1232
Training samples: 1091
Validation samples: 183
Testing samples: 546
Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 183s 4s/step - accuracy: 0.5691 - loss: 1.0226 - val_accuracy: 0.6940 - val_loss: 0.5287
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 424ms/step - accuracy: 0.6906 - loss: 0.5770 - val_accuracy: 0.7814 - val_loss: 0.5026
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 426ms/step - accuracy: 0.7743 - loss: 0.5098 - val_accuracy: 0.7705 - val_loss: 0.4497
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 575ms/step - accuracy: 0.7780 - loss: 0.4774 - val_accuracy: 0.8087 - val_loss: 0.4112
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 422ms/step - accuracy: 0.8133 - loss: 0.4272 - val_accuracy: 0.8306 - val_loss: 0.3851
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 499ms/step - accuracy: 0.8001 - loss: 0.4192 - val_accuracy: 0.8415 - val_loss: 0.3791
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 540ms/step - accuracy: 0.8478 - loss: 0.3697 - val_accuracy: 0.8470 - v

# Here is a table summarizing the performance metrics for all magnification factors (40X, 100X, 200X, 400X) from notebook:

In [1]:
import pandas as pd

# Create a DataFrame with the provided values
data = {
    'Magnification': ['40X', '100X', '200X', '400X'],
    'Test Accuracy': [0.8748, 0.8498, 0.8411, 0.8352],
    'F1 Score': [0.9100, 0.8929, 0.8873, 0.8822],
    'G-Mean': [0.8433, 0.8081, 0.7938, 0.7847],
    'Informedness (IBA)': [0.6934, 0.6275, 0.6017, 0.5869]
}

df = pd.DataFrame(data)

# Display the table with formatting
styled_df = df.style \
    .format({
        'Test Accuracy': '{:.4f}',
        'F1 Score': '{:.4f}',
        'G-Mean': '{:.4f}',
        'Informedness (IBA)': '{:.4f}'
    }) \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#000000'), ('font-weight', 'bold'), ('color', 'white')]}
    ]) \
    .hide(axis='index')

styled_df


Magnification,Test Accuracy,F1 Score,G-Mean,Informedness (IBA)
40X,0.8748,0.9100,0.8433,0.6934
100X,0.8498,0.8929,0.8081,0.6275
200X,0.8411,0.8873,0.7938,0.6017
400X,0.8352,0.8822,0.7847,0.5869
